In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd

In [6]:
from ast import literal_eval
train = pd.read_csv('/content/drive/MyDrive/NLP_Project/train.csv', encoding='utf-8', converters={'clean_body': literal_eval})
test = pd.read_csv('/content/drive/MyDrive/NLP_Project/test.csv', encoding='utf-8', converters={'clean_body': literal_eval})

In [2]:
import re
from tqdm import tqdm_notebook

from nltk.corpus import stopwords

from tensorflow.keras import regularizers, initializers, optimizers, callbacks
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

MAX_NB_WORDS = 100000    # max no. of words for tokenizer
MAX_SEQUENCE_LENGTH = 200 # max length of each entry (sentence), including padding
VALIDATION_SPLIT = 0.2   # data for validation (not used in training)
EMBEDDING_DIM = 100      # embedding dimensions for word vectors (word2vec/GloVe)
GLOVE_DIR = "glove/glove.6B."+str(EMBEDDING_DIM)+"d.txt"

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tqdm import tqdm

In [3]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-04-22 20:31:21--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-04-22 20:31:21--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-22 20:31:21--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [7]:
# Create X_train, X_test, y_train , y_test

X_train = pd.DataFrame(train['clean_body'])
y_train = train['category']

X_test = pd.DataFrame(test['clean_body'])
y_test =test['category']

In [8]:
X = pd.concat([X_train, X_test], axis = 0)
X

,clean_body
0,"[sent, thursday, blocked, mails, hello, collea..."
1,"[wednesday, july, pm, mailbox, almost, importa..."
2,"[opportunity, code, hi, please, add, opportuni..."
3,"[thursday, pm, removal, hello, decommissioned,..."
4,"[sent, monday, timecard, hello, hours, set, au..."
...,...
14560,"[question, job, dear, please, message, bellow,..."
14561,"[friday, pm, ad, groups, rename, hello, please..."
14562,"[updated, bandwidth, upgrade, hello, please, k..."
14563,"[wednesday, pm, questions, dear, several, ques..."


In [9]:
Y= pd.concat([y_train, y_test], axis = 0)
Y

0        4
1        4
2        4
3        4
4        4
        ..
14560    4
14561    4
14562    6
14563    4
14564    4
Name: category, Length: 111091, dtype: int64

In [14]:
embedding_vector = {}
f = open('glove.6B.300d.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

400000it [00:27, 14336.78it/s]


In [18]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(X['clean_body'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 17599 unique tokens.


In [19]:
X = tokenizer.texts_to_sequences(X['clean_body'])
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)
X

Shape of data tensor: (111091, 250)


array([[   0,    0,    0, ...,  151, 1779,   48],
       [   0,    0,    0, ...,  704,   55,   19],
       [   0,    0,    0, ...,   47, 2093, 3704],
       ...,
       [   0,    0,    0, ...,  146,    5,   84],
       [   0,    0,    0, ...,  307,    5,  366],
       [   0,    0,    0, ...,    1,   20,    5]], dtype=int32)

In [23]:
vocab_size=len(word_index)+1
embedding_matrix = np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 17599/17599 [00:00<00:00, 419020.88it/s]


In [26]:
Y = pd.get_dummies(Y).values
print('Shape of label tensor:', Y.shape)
Y

Shape of label tensor: (111091, 4)


array([[0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       ...,
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 0]], dtype=uint8)

In [27]:
X_train = X[:len(X_train)]
X_test = X[len(X_train):]
y_train = Y[:len(y_train)]
y_test = Y[len(y_train):]

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(96526, 250) (96526, 4)
(14565, 250) (14565, 4)


In [29]:
model = Sequential()
model.add(Embedding(vocab_size, 300,weights=[embedding_matrix], input_length=X.shape[1], trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2,unroll=True))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 300)          5280000   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 250, 300)         0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 4)                 404       
                                                                 
Total params: 5,440,804
Trainable params: 160,804
Non-trainable params: 5,280,000
_________________________________________________________________
None


In [30]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=7, batch_size=128)

Epoch 1/7
755/755 [==============================] - 305s 257ms/step - loss: 0.8611 - accuracy: 0.6627 - val_loss: 0.5953 - val_accuracy: 0.7862
Epoch 2/7
755/755 [==============================] - 180s 238ms/step - loss: 0.6464 - accuracy: 0.7587 - val_loss: 0.5470 - val_accuracy: 0.8084
Epoch 3/7
755/755 [==============================] - 180s 238ms/step - loss: 0.5539 - accuracy: 0.7952 - val_loss: 0.5343 - val_accuracy: 0.8093
Epoch 4/7
755/755 [==============================] - 177s 235ms/step - loss: 0.4915 - accuracy: 0.8182 - val_loss: 0.4523 - val_accuracy: 0.8373
Epoch 5/7
755/755 [==============================] - 177s 234ms/step - loss: 0.4463 - accuracy: 0.8356 - val_loss: 0.3878 - val_accuracy: 0.8604
Epoch 6/7
755/755 [==============================] - 175s 232ms/step - loss: 0.4107 - accuracy: 0.8478 - val_loss: 0.3772 - val_accuracy: 0.8635
Epoch 7/7
755/755 [==============================] - 174s 231ms/step - loss: 0.3827 - accuracy: 0.8586 - val_loss: 0.3801 - val_ac